1. 학습데이터, 테스트데이터 만들기
먼저 모델을 만들고 나서 학습을 시킬 데이터를 만들기 위해 네이버에서 몇몇의 기업을 선정하고

그 기업에 대한 기사의 제목을 크롤링하였습니다.

 

크롤링 후에 학습시 필요한 긍정, 부정, 중립을 나타내는 label이 있었어야했는데

손으로 일일이 부정, 긍정, 중립 이 세 가지로 label을 붙이려고 하다보니

눈도 아프고 비슷한 내용임에도 불구하고 앞에서는 긍정으로 했다가 뒤에서는 중립으로 표기하는 등의 문제가 있었습니다.

 

이를 컴퓨터가 긍정적인 단어, 부정적인 단어가 포함되어있는지 여부를 확인하여

자동으로 라벨을 붙여주면 편할 것 같아 그렇게 만들어 보았습니다.

 

먼저, 긍정적인 단어, 부정적인 단어가 포함된 txt파일을 각각 만들어주었습니다.

negative_words_self.txt
0.00MB
positive_words_self.txt
0.00MB
뉴스 기사를 보며 만든 긍정적인 단어, 부정적인 단어 모음입니다.

단어는 생각나는대로 계속 추가하고자합니다.

 

코드에서는 이 단어들을 파일에서 positive, negative라는 list로 받아와서

두 개의 list를 합쳐 posneg라는 list를 만들고 크롤링해오는 단어에서 posneg안에 있는 단어가 포함되어있으면 긍정, 부정 라벨을 붙여주고

포함되어있지 않으면 그냥 중립인 0의 상태로 그대로 두도록 만들어 보았습니다.

 

파일에서 단어를 불러와 posneg리스트를 만드는 코드

In [15]:
def category_to_dataset(my_title_dic, category, length):
    import codecs

    keywords = []

    file = codecs.open("./" + category + ".txt", 'rb', encoding='UTF-8')

    while True:
      line = file.readline()
      line = line.replace('\n', '')
      keywords.append(line)

      if not line: break
    file.close()

    import requests
    from bs4 import BeautifulSoup
    import re
    import pandas as pd

    for i in range(100):#
      print("crawling progress: ",category,"[",i,"]")
      num = 1 - 10 + 10 * 120#
      for j in range(length // 100 // 10):
          num = num + 10
          # bhc
        # url = "https://search.naver.com/search.naver?&where=news&query=bhc&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=23&start=" + str(num)
          # 아오리라멘
        # url2 = "https://search.naver.com/search.naver?&where=news&query=%EC%95%84%EC%98%A4%EB%A6%AC%EB%9D%BC%EB%A9%98&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=34&start=" + str(num)

          # 버거킹
          url3 = "https://search.naver.com/search.naver?&where=post&query="+keywords[i]+"&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=23&start=" + str(num)

          req = requests.get(url3)

          soup = BeautifulSoup(req.text, 'lxml')

          titles = soup.select("a._sp_each_title")
            
          counter=0
          for title in titles:

            title_data = title.text
            title_data = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“》]', '', title_data)
            my_title_dic['title'].append(title_data)
            my_title_dic['label'].append(category)

            counter=counter+1
        
          if(counter<10):
                print("Cannot query ",i,"th keyword anymore.")
                break

In [16]:
def dftoCsv(my_title_dic, num):
  import pandas as pd
  
  my_title_df = pd.DataFrame(my_title_dic)
  my_title_df.to_csv(('./title_datas'+ str(num) +'.csv'), sep=',', na_rep='NaN', encoding='utf-8')

In [20]:
def get_sentiment_dataset(length):
    my_title_dic = {"title":[], "label":[]}
    
    category_to_dataset(my_title_dic, "긍정", length//2)
    category_to_dataset(my_title_dic, "부정", length//2)
    
    dftoCsv(my_title_dic,1)

In [21]:
def get_category_dataset(length):
    my_title_dic = {"title":[], "label":[]}
    
    category_to_dataset(my_title_dic, "IT", length//6)
    category_to_dataset(my_title_dic, "건강", length//6)
    category_to_dataset(my_title_dic, "경제", length//6)
    category_to_dataset(my_title_dic, "뷰티", length//6)
    category_to_dataset(my_title_dic, "엔터테인먼트", length//6)
    category_to_dataset(my_title_dic, "생활", length//6)
    
    dftoCsv(my_title_dic,1)

In [22]:
get_sentiment_dataset(2*100*10*30)

crawling progress:  긍정 [ 0 ]
crawling progress:  긍정 [ 1 ]
crawling progress:  긍정 [ 2 ]
crawling progress:  긍정 [ 3 ]
crawling progress:  긍정 [ 4 ]
crawling progress:  긍정 [ 5 ]
crawling progress:  긍정 [ 6 ]
crawling progress:  긍정 [ 7 ]
crawling progress:  긍정 [ 8 ]
crawling progress:  긍정 [ 9 ]
crawling progress:  긍정 [ 10 ]
crawling progress:  긍정 [ 11 ]
crawling progress:  긍정 [ 12 ]
crawling progress:  긍정 [ 13 ]
crawling progress:  긍정 [ 14 ]
crawling progress:  긍정 [ 15 ]
crawling progress:  긍정 [ 16 ]
crawling progress:  긍정 [ 17 ]
crawling progress:  긍정 [ 18 ]
crawling progress:  긍정 [ 19 ]
crawling progress:  긍정 [ 20 ]
crawling progress:  긍정 [ 21 ]
crawling progress:  긍정 [ 22 ]
crawling progress:  긍정 [ 23 ]
crawling progress:  긍정 [ 24 ]
crawling progress:  긍정 [ 25 ]
crawling progress:  긍정 [ 26 ]
crawling progress:  긍정 [ 27 ]
crawling progress:  긍정 [ 28 ]
crawling progress:  긍정 [ 29 ]
crawling progress:  긍정 [ 30 ]
crawling progress:  긍정 [ 31 ]
crawling progress:  긍정 [ 32 ]
crawling progress:  